In [1]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

In [2]:
from src.select_functions import *

### 1. Connecting with Foursquare Api.

In [3]:
load_dotenv()
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [4]:
office_location = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}
print (office_location)

{'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}


### 2. Defining Starbucks' query

In [5]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [6]:
parameters = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"Starbucks",
              "radius":500}

In [7]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)
list(data)

['meta', 'response']

In [8]:
decoding_data = data.get("response")
decoded = decoding_data.get("groups")[0]
starbucks = decoded.get("items")

In [9]:
map_ = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [10]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [11]:
print(getFromDict(starbucks[0],map_))

Starbucks


In [12]:
starbucks_nyc = []
for dic in starbucks:
    list1 = {}
    list1["name"] = getFromDict(dic,map_)
    list1["latitud"] = getFromDict(dic,m_latitude)
    list1["longitud"] = getFromDict(dic,m_longitude)
    starbucks_nyc.append(list1)

In [13]:
# Conversion on dataframe that are used on maps construction.
df_starbucks = pd.DataFrame(starbucks_nyc)
df_starbucks.head()

,name,latitud,longitud
0,Starbucks,40.758149,-73.979420
1,Starbucks,40.759535,-73.980981
2,Starbucks,40.760602,-73.976467
3,Starbucks,40.761845,-73.978700
4,Starbucks,40.756665,-73.974397


#### A sum of 20 Starbucks are located in a radius of 500m from the office.


In [14]:
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud,df_starbucks.latitud))

In [15]:
### Creating the column criteria
gdf_starbucks = gdf_starbucks.assign(name = [ "Starbucks"]*20,
               criteria = ["Starbucks"]*20)
gdf_starbucks.head()

,name,latitud,longitud,geometry,criteria
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815),Starbucks
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953),Starbucks
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060),Starbucks
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185),Starbucks
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666),Starbucks


In [16]:
# Convert our dataframe on a dict to be able to use it in MongoDB.

In [17]:
gdf_starbucks["geometry"] = gdf_starbucks["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_starbucks

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Starbucks,40.758149,-73.979420,"{'type': 'Point', 'coordinates': (-73.97941981...",Starbucks
1,Starbucks,40.759535,-73.980981,"{'type': 'Point', 'coordinates': (-73.98098084...",Starbucks
2,Starbucks,40.760602,-73.976467,"{'type': 'Point', 'coordinates': (-73.97646651...",Starbucks
3,Starbucks,40.761845,-73.978700,"{'type': 'Point', 'coordinates': (-73.9787, 40...",Starbucks
4,Starbucks,40.756665,-73.974397,"{'type': 'Point', 'coordinates': (-73.97439689...",Starbucks
5,Starbucks,40.759621,-73.973656,"{'type': 'Point', 'coordinates': (-73.97365629...",Starbucks
6,Starbucks,40.756460,-73.978353,"{'type': 'Point', 'coordinates': (-73.97835262...",Starbucks
7,Starbucks,40.760725,-73.978769,"{'type': 'Point', 'coordinates': (-73.9787691,...",Starbucks
8,Starbucks,40.760567,-73.983809,"{'type': 'Point', 'coordinates': (-73.98380932...",Starbucks
9,Starbucks,40.756086,-73.974971,"{'type': 'Point', 'coordinates': (-73.97497083...",Starbucks


In [18]:
type(gdf_starbucks.loc[0,"geometry"])

dict

### Export the query to Mongo

In [19]:
client = MongoClient() 
db = client.companies
collection = db.starbucks 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [20]:
data = gdf_starbucks.to_dict(orient='records')
data[0]

{'name': 'Starbucks',
 'latitud': 40.75814852475969,
 'longitud': -73.97941981951507,
 'geometry': {'type': 'Point',
  'coordinates': (-73.97941981951507, 40.75814852475969)},
 'criteria': 'Starbucks'}

In [21]:
collection.insert_many(data)

### Determine the nearest Starbucks

In [ ]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.starbucks.find(filt,proyect).limit(1)
starbucks = list(result)

In [ ]:
starbucks